## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-02-21 15:48:52,814 - root - INFO - Logger set
2019-02-21 15:48:52,815 - root - INFO - Hostname zinch


### Session parameters and raw files

In [2]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

# sess_par = {'bird': 'p14r14',
#            'sess': '2019-02-14_2250_01'}
sess_par = {'bird': 'p14r14',
           'sess': '2019-02-15_3125_01'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'])

logger.info('Raw folder {}'.format(exp_struct['folders']['raw']))
exp_struct

# get the rig parametrs
## Convert the whole session to a kwd file
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']

bin_path = exp_struct['files']['mda_raw']


2019-02-21 15:49:00,536 - root - INFO - Raw folder /mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01


In [3]:
exp_struct

{'folders': {'raw': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01',
  'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01',
  'msort': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_01'},
 'files': {'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_01/params.json',
  'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01/settings.isf',
  'rig': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01/rig.json',
  'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/streams.kwd',
  'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/spikes.kwik',
  'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/events.kwe',
  'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_01/raw.mda'}}

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [4]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path)

2019-02-21 15:49:08,766 - intan2kwik.kwd - INFO - reading intan chans data across all of rec /mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01
2019-02-21 15:49:08,832 - intan2kwik.kwd - INFO - Found 5 .rhd files split in 5 recordings
2019-02-21 15:49:08,833 - intan2kwik.kwd - INFO - dest file: /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/streams.kwd


2019-02-21 15:50:58,221 - intan2kwik.kwd - WARNING - Failed to read file /mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01/raw_190215_103028.rhd, maybe it is corrupted
2019-02-21 15:50:58,224 - intan2kwik.kwd - WARNING - Could not read a file in rec {}, skipping the file group


2019-02-21 15:51:01,471 - intan2kwik.kwd - WARNING - Failed to read file /mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01/raw_190215_103455.rhd, maybe it is corrupted
2019-02-21 15:51:01,473 - intan2kwik.kwd - WARNING - Could not read a file in rec {}, skipping the file group


2019-02-21 15:52:21,019 - intan2kwik.kwd - INFO - moving back to /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/streams.kwd
2019-02-21 15:52:25,311 - intan2kwik.kwd - INFO - removing temp file


#### Update a session with subsequently recorded rhd files

In [5]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-02-21 15:52:25,321 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_01/streams.kwd from folder /mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-15_3125_01


KeyError: "Unable to open object (object 'data' doesn't exist)"

### Kwd file to mda 

## Select a subset of a kwd file:
 - a set of recordings (e.g, by range of time)
 - a set of channels (by identifier) (e.g: a whole port from the intan, like 'A-', or the channel where the mic is, like 'ADC-00')

In [ ]:
pd_meta = kwdf.get_all_rec_meta(exp_struct['files']['kwd'])

In [ ]:
reload(kwdf)
pd_meta = kwdf.get_all_rec_meta(kwd_path)

# Select channels:
wanted_chans = np.array(['A-', 'ADC'])

all_chan_names = kwdf.get_all_chan_names(pd_meta)

sel_chan_names = kwdf.get_all_chan_names(pd_meta, chan_filt=wanted_chans)

#in this case, that's all the channels
sel_chan_names

In [ ]:
wanted_chans = np.array(['A-']) # all ephys channels
chan_list = kwdf.get_all_chan_names(pd_meta, chan_filt=wanted_chans)

## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [ ]:
exp_struct['folders']

In [12]:
reload(kwdf)
reload(kutil)
ch.setLevel(logging.INFO)

all_rec_list = kutil.get_rec_list(kwd_path)
all_rec_list

wanted_chans = np.array(['A-']) # all ephys channels
chan_list = kwdf.get_all_chan_names(pd_meta, chan_filt=wanted_chans)

bin_path = exp_struct['files']['mda_raw']
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)
bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'], exp_struct['files']['mda_raw'], chan_list=chan_list, rec_list=all_rec_list, header='mda')
rec_attr_dict = kwdf.get_rec_attrs(exp_struct['files']['kwd'], 0)
all_rec_meta_pd = kwdf.get_all_rec_meta(exp_struct['files']['kwd'])


2019-02-20 17:46:39,709 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/streams.kwd to binary
2019-02-20 17:46:39,769 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: ['A-000' 'A-001' 'A-002' 'A-003' 'A-004' 'A-005' 'A-006' 'A-007' 'A-008'
 'A-009' 'A-010' 'A-011' 'A-012' 'A-013' 'A-014' 'A-015' 'A-016' 'A-017'
 'A-018' 'A-019' 'A-020' 'A-021' 'A-022' 'A-023' 'A-024' 'A-025' 'A-026'
 'A-027' 'A-028' 'A-029' 'A-030' 'A-031']
2019-02-20 17:46:39,769 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53]
2019-02-20 17:46:39,770 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Creating binary file /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda


2019-02-20 17:48:31,240 - pipefinch.h5tools.kwik.kwdfunctions - INFO - 5609088000 elements written


#### test that the mda was written ok by looking at the microphone channel

In [13]:
# from mountainlab_pytools import mdaio
# loaded_mda = mdaio.readmda(bin_path)
# start = 8400000 + 30000
# end = start + 4000
# plt.plot(loaded_mda[:, start:end].T);

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [6]:
reload(msc)
file_paths, out_folder = msc.make_paths(exp_struct['folders']['msort'])
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

#get the rig parametrs
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe']['probe_0']['model']
headstage = rig_par['probe']['probe_0']['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)
probe_geom = probes.make_map(probe, probe_chans)
np.savetxt(file_paths['geom'], probe_geom, delimiter=',', fmt='%d')


# parameters to pass to the msort scripts
ds_params.update({'samplerate': int(kwdf.get_sampling_rate(all_rec_meta_pd, 0)), # required
                 })

# parameters to override defaults of msc.sort_dataset()



with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


NameError: name 'pd_meta' is not defined

In [24]:
np.unique(probe_geom.T[1])

array([  0,  20,  40,  60,  80, 100, 120, 140, 160, 180, 200, 220, 240,
       260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500,
       520, 540, 560, 580, 600, 620])

In [25]:
ds_params

{'samplerate': 20000, 'detect_sign': -1}

In [26]:
file_paths

{'mda': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda',
 'params': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/params.json',
 'geom': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/geom.csv',
 'filt': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv',
 'pre': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv',
 'firings': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda',
 'firings_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics_curated.json'}

In [27]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(exp_struct['folders']['msort'])

{'samplerate': 20000, 'detect_sign': -1}

In [28]:
# Initialize a pipeline
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline();

JSProxyWidget(status='Not yet rendered')

In [29]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=600, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={}, no_auto_metrics=False)
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [30]:
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, **sort_params);

2019-02-21 12:51:10,342 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv


2019-02-21 12:51:14,773 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 55f1e2c1896ab620259f10822882c7b7836f3cec
[ Checking outputs... ]
{"timeseries_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv
false
{"timeseries_out":"/media/zinch/Windows/experiment/msort-tmp/output_55f1e2c1896ab620259f10822882c7b7836f3cec_timeseries_out.mda"}
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ephys.whiten --inputs timeseries:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv --parameters --outputs timeseries_out:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv


2019-02-21 12:51:48,058 - pipefinch.sort.mountain.comre - INFO - Sorting


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 686b9f7e40d124bfb7b26e19ba3892b0b6b5755e
[ Checking outputs... ]
{"timeseries_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv
false
{"timeseries_out":"/media/zinch/Windows/experiment/msort-tmp/output_686b9f7e40d124bfb7b26e19ba3892b0b6b5755e_timeseries_out.mda"}
[ Checking process cache ... ]
[ Process ephys.whiten already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ms4alg.sort --inputs geom:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/geom.csv timeseries:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv --parameters adjacency_radius:100 detect_sign:-1 detect_threshold:2 --outputs firin

2019-02-21 12:51:52,020 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 92a2ab40af3d84f1016fb4a49bc809938c2fcfc8
[ Checking outputs... ]
{"firings_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda
false
{"firings_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Process ms4alg.sort already completed. ]
[ Done. ]
RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda timeseries:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv --parameters samplerate:20000 --outputs cluster_metrics_out:/media/zinch/Windows/experiment/msort-tmp/mountainlab/tmp_short_term/out

2019-02-21 13:01:47,329 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: b84d234ff66716184fc67ae750a0487a5819dced
[ Checking outputs... ]
{"metrics_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json
false
{"metrics_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/media/zinch/Windows/experiment/m

In [32]:
file_paths

{'mda': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda',
 'params': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/params.json',
 'geom': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/geom.csv',
 'filt': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/filt.mda.prv',
 'pre': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/pre.mda.prv',
 'firings': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings.mda',
 'firings_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

#### export kwik file

In [48]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'firings_curated' or 'firings' for default_output


metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save])


2019-02-21 15:42:55,753 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/spikes.kwik from firings /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/sort_out/firings_curated.mda


In [36]:
### extract all unit waveforms
from pipefinch.neural import units
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'])

2019-02-21 13:37:14,250 - pipefinch.neural.units - INFO - About to get all waveforms for 39 units in file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/spikes.kwik


0

In [50]:
def msort_cleanup(exp_struct: dict):
    # remove the mda files and try to cleanup the msort temp location
    mda_raw_path = exp_struct['files']['mda_raw']
    logger.info('removing intermediate msort mda file {}'.format(mda_raw_path))
    os.remove(mda_raw_path)

def msort_tmp_clean():
    tmp_dir = os.path.abspath(os.environ['ML_TEMPORARY_DIRECTORY'])
    logger.info('Cleaning up msort temp dir {}'.format(tmp_dir))
    
#msort_tmp_clean()
msort_cleanup(exp_struct)

2019-02-21 15:43:21,978 - root - INFO - removing intermediate msort mda file /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda


In [40]:
exp_struct['files']

{'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/params.json',
 'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-14_2250_02/settings.isf',
 'rig': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-14_2250_02/rig.json',
 'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/streams.kwd',
 'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/spikes.kwik',
 'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-14_2250_02/events.kwe',
 'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-14_2250_02/raw.mda'}

In [41]:
shutil.rmtree?

Signature: shutil.rmtree(path, ignore_errors=False, onerror=None)
Docstring:
Recursively delete a directory tree.

If ignore_errors is set, errors are ignored; otherwise, if onerror
is set, it is called to handle the error with arguments (func,
path, exc_info) where func is platform and implementation dependent;
path is the argument to that function that caused it to fail; and
exc_info is a tuple returned by sys.exc_info().  If ignore_errors
is false and onerror is None, an exception is raised.
File:      ~/conda/envs/msort/lib/python3.6/shutil.py
Type:      function


In [47]:
os.environ

environ{'STY': '3043..zinch',
        'TERM': 'xterm-color',
        'TERMCAP': 'SC|screen|VT 100/ANSI X3.64 virtual terminal:\\\n\t:DO=\\E[%dB:LE=\\E[%dD:RI=\\E[%dC:UP=\\E[%dA:bs:bt=\\E[Z:\\\n\t:cd=\\E[J:ce=\\E[K:cl=\\E[H\\E[J:cm=\\E[%i%d;%dH:ct=\\E[3g:\\\n\t:do=^J:nd=\\E[C:pt:rc=\\E8:rs=\\Ec:sc=\\E7:st=\\EH:up=\\EM:\\\n\t:le=^H:bl=^G:cr=^M:it#8:ho=\\E[H:nw=\\EE:ta=^I:is=\\E)0:\\\n\t:li#24:co#80:am:xn:xv:LP:sr=\\EM:al=\\E[L:AL=\\E[%dL:\\\n\t:cs=\\E[%i%d;%dr:dl=\\E[M:DL=\\E[%dM:dc=\\E[P:DC=\\E[%dP:\\\n\t:im=\\E[4h:ei=\\E[4l:mi:IC=\\E[%d@:ks=\\E[?1h\\E=:\\\n\t:ke=\\E[?1l\\E>:vi=\\E[?25l:ve=\\E[34h\\E[?25h:vs=\\E[34l:\\\n\t:ti=\\E[?1049h:te=\\E[?1049l:Km=\\E[M:k0=\\E[10~:k1=\\EOP:\\\n\t:k2=\\EOQ:k3=\\EOR:k4=\\EOS:k5=\\E[15~:k6=\\E[17~:k7=\\E[18~:\\\n\t:k8=\\E[19~:k9=\\E[20~:k;=\\E[21~:F1=\\E[23~:F2=\\E[24~:\\\n\t:kh=\\E[1~:@1=\\E[1~:kH=\\E[4~:@7=\\E[4~:kN=\\E[6~:kP=\\E[5~:\\\n\t:kI=\\E[2~:kD=\\E[3~:ku=\\EOA:kd=\\EOB:kr=\\EOC:kl=\\EOD:',
        'WINDOW': '0',
        'SHELL': '/bin/bash'